# KNN

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [19]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[["League", "Division", "NewLeague"]])

y = df["Salary"]
X_ = df.drop(["Salary", "League", "Division", "NewLeague"], axis=1).astype("float64")

X = pd.concat([X_, dms[["League_N", "Division_W", "NewLeague_N"]]], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=25,
                                                   random_state=42)

In [25]:
# KNN model
knn_model = KNeighborsRegressor()

In [26]:
# Model fit etme işlemi
knn_model.fit(X_train, y_train)

KNeighborsRegressor()

## Tahmin

In [28]:
# Test veri kümesinde tahmin
y_pred = knn_model.predict(X_test)

In [30]:
np.sqrt(mean_squared_error(y_test, y_pred))

360.6113043414851

In [34]:
# k değerlerinin RMSE değerleri

k_values = list(range(1, 11))# 1'den 10'e kadar K değerleri
RMSE = [] # RMSE değerlerini saklayacağımız bir liste

for k in k_values:
    knn_model = KNeighborsRegressor(n_neighbors=k)# KNN modelini oluştur
    knn_model.fit(X_train, y_train)  # Modeli eğit
    y_pred = knn_model.predict(X_test)# Test veri kümesinde tahmin yap
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))# RMSE değerini hesapla
    RMSE.append(rmse)# RMSE değerini listeye ekle
    print(f"K={k} için RMSE: {rmse}")# Sonuçları yazdır


K=1 için RMSE: 393.72755857150764
K=2 için RMSE: 345.81929108855394
K=3 için RMSE: 327.0130272296877
K=4 için RMSE: 365.30132526069485
K=5 için RMSE: 360.6113043414851
K=6 için RMSE: 351.92107440471176
K=7 için RMSE: 363.21239379640656
K=8 için RMSE: 359.0692782925671
K=9 için RMSE: 353.84000797538533
K=10 için RMSE: 344.73595938380146


In [35]:
print("K Değerleri: ", k_values)
print("RMSE Değerleri: ", RMSE)

K Değerleri:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
RMSE Değerleri:  [393.72755857150764, 345.81929108855394, 327.0130272296877, 365.30132526069485, 360.6113043414851, 351.92107440471176, 363.21239379640656, 359.0692782925671, 353.84000797538533, 344.73595938380146]


## KNN Model Tuning

In [43]:
# GridSearchCV ifadesini kullanabilmek için 1-30 arasında n_neighbors
# ismiyle knn parametresi oluşturduk
knn_params = {'n_neighbors': np.arange(1, 30, 1)}

In [44]:
knn = KNeighborsRegressor()

In [45]:
knn_cv_model = GridSearchCV(knn, knn_params, cv=10)

In [46]:
knn_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])})

In [48]:
# En iyi knn parametresi RMSE değeri için
knn_cv_model.best_params_["n_neighbors"]

3

In [49]:
# En iyi knn sayısı ile model kurumu
knn_tuned = KNeighborsRegressor(n_neighbors = knn_cv_model.best_params_["n_neighbors"])

In [50]:
knn_tuned.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [52]:
# Tuned edilmiş hata değeri
np.sqrt(mean_squared_error(y_test, knn_tuned.predict(X_test)))

327.0130272296877